<a href="https://colab.research.google.com/github/Govamsh/AWS_test/blob/main/retail_image_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# New section

In [ ]:
import cv2
import imghdr
import os
from matplotlib import pyplot as plt

data_dir='/content/drive/MyDrive/ListMissingProducts/Shelf1'

image_exts=['jpeg','jpg','bmp','png']


#1.2 Remove dodgy images

for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir,image_class)):
    image_path = os.path.join(data_dir,image_class,image)
    try:
      img=cv2.imread(image_path)
      tip=imghdr.what(image_path)
      if tip not in image_exts:
        print('Image not in ext list{}'.format(image_path))
        os.remove(image_path)
    except Exception as e:
      print('Issue with image {}'.format(image_path))
      os.remove(image_path)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os
import matplotlib.pyplot as plt

data_dir='/content/drive/MyDrive/ListMissingProducts/Shelf1'
class_labels = []
for image_class in os.listdir(data_dir):
    class_labels+=[image_class]

print(class_labels)


['monster_energy_ultra', 'monster_energy_pipeline', 'monster_energy_ultra_fiesta_mango', 'monster_energy_juice_mango_loco', 'monster_energy_original', 'monster_energy_juice_monarch', 'monster_energy_zero_sugar_taurine', 'monster_energy_juice_aussie_lemonade', 'monster_energy_lewis_hamilton', 'monster_energy_ultra_peachy_keen', 'monster_energy_nitro', 'monster_energy_pacific_punch', 'monster_energy_white_pineapple', 'monster_energy_mixxd', 'monster_energy_ultra_rosa']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = '/content/drive/MyDrive/ListMissingProducts/Shelf1'

# Define ImageDataGenerator for preprocessing and loading images
image_size = (224, 224)  # Update with your desired image size
batch_size = 32  # Adjust batch size as needed

# Using ImageDataGenerator for data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values between 0 and 1
    validation_split=0.2  # Split data into training and validation sets
)

# Load and prepare the dataset using flow_from_directory
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use categorical mode for multiple classes
    subset='training'  # Specify that this is the training set
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify that this is the validation set
)

# Define and compile your model (Example: using a simple CNN)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Update with desired number of epochs
    validation_data=validation_generator
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(accuracy * 100))


Found 119 images belonging to 15 classes.
Found 24 images belonging to 15 classes.
Epoch 1/10
4/4 [==============================] - 16s 3s/step - loss: 37.9827 - accuracy: 0.0672 - val_loss: 30.6766 - val_accuracy: 0.1250
Epoch 2/10
4/4 [==============================] - 17s 3s/step - loss: 24.7834 - accuracy: 0.1092 - val_loss: 14.0322 - val_accuracy: 0.1667
Epoch 3/10
4/4 [==============================] - 14s 3s/step - loss: 10.2135 - accuracy: 0.2437 - val_loss: 4.5137 - val_accuracy: 0.3333
Epoch 4/10
4/4 [==============================] - 13s 3s/step - loss: 3.0917 - accuracy: 0.3613 - val_loss: 2.7913 - val_accuracy: 0.1667
Epoch 5/10
4/4 [==============================] - 13s 3s/step - loss: 1.9215 - accuracy: 0.4370 - val_loss: 2.4168 - val_accuracy: 0.2917
Epoch 6/10
4/4 [==============================] - 12s 3s/step - loss: 1.2291 - accuracy: 0.6639 - val_loss: 2.5542 - val_accuracy: 0.2917
Epoch 7/10
4/4 [==============================] - 14s 4s/step - loss: 0.8260 - accur

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = '/content/drive/MyDrive/ListMissingProducts/Shelf1'

# Define ImageDataGenerator for preprocessing and loading images
image_size = (224, 224)
batch_size = 32

# Using ImageDataGenerator for data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Using a pre-trained model as the base (example with MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(accuracy * 100))


Found 119 images belonging to 15 classes.
Found 24 images belonging to 15 classes.
Epoch 1/10
4/4 [==============================] - 18s 3s/step - loss: 3.2063 - accuracy: 0.0840 - val_loss: 2.2481 - val_accuracy: 0.2500
Epoch 2/10
4/4 [==============================] - 13s 3s/step - loss: 2.3334 - accuracy: 0.2437 - val_loss: 1.9561 - val_accuracy: 0.3750
Epoch 3/10
4/4 [==============================] - 17s 4s/step - loss: 1.7063 - accuracy: 0.4706 - val_loss: 1.5570 - val_accuracy: 0.6667
Epoch 4/10
4/4 [==============================] - 12s 3s/step - loss: 1.3303 - accuracy: 0.5714 - val_loss: 1.4963 - val_accuracy: 0.5833
Epoch 5/10
4/4 [==============================] - 14s 4s/step - loss: 1.0481 - accuracy: 0.6555 - val_loss: 1.6479 - val_accuracy: 0.4583
Epoch 6/10
4/4 [==============================] - 14s 4s/step - loss: 0.8567 - accuracy: 0.7395 - val_loss: 1.3048 - val_accuracy: 0.6250
Epoch 7/10
4/4 [==============================] - 17s 4s/step - loss: 0.7059 - accuracy: 

In [ ]:

# Unfreeze the last few layers for fine-tuning
for layer in base_model.layers[:-10]:
    layer.trainable = True

# Implement learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
)

# Compile the model with a lower learning rate and momentum optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Retrain the model with fine-tuning
history = model.fit(
    train_generator,
    epochs=5,  # Increase epochs for fine-tuning
    validation_data=validation_generator
)

Epoch 1/5
4/4 [==============================] - 62s 8s/step - loss: 1.2930 - accuracy: 0.6134 - val_loss: 1.0821 - val_accuracy: 0.6667
Epoch 2/5
4/4 [==============================] - 30s 7s/step - loss: 0.8132 - accuracy: 0.7311 - val_loss: 1.1926 - val_accuracy: 0.5417
Epoch 3/5
4/4 [==============================] - 34s 8s/step - loss: 0.6981 - accuracy: 0.7647 - val_loss: 1.1116 - val_accuracy: 0.5417
Epoch 4/5
4/4 [==============================] - 31s 7s/step - loss: 0.3856 - accuracy: 0.8824 - val_loss: 1.3591 - val_accuracy: 0.4583
Epoch 5/5
4/4 [==============================] - 32s 7s/step - loss: 0.3706 - accuracy: 0.8655 - val_loss: 1.4126 - val_accuracy: 0.5000


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to your test image
test_image_path = '/content/drive/MyDrive/ListMissingProducts/test_miss.jpeg'  # Replace with your image path

# Load the image and resize it to match the input size used during training
img = image.load_img(test_image_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand the dimensions to create a batch as the model expects batches of images
img_batch = np.expand_dims(img_array, axis=0)

# Preprocess the image (similar to how it was done during training)
img_preprocessed = img_batch / 255.0  # Rescaling

# Make predictions using the trained model
predictions = model.predict(img_preprocessed)

# Get the class labels
class_indices = train_generator.class_indices

# Inverse mapping of indices to class labels
inverse_mapping = {v: k for k, v in class_indices.items()}

# Get the predicted class label
predicted_class = inverse_mapping[np.argmax(predictions)]

print(f"The predicted class for the test image is: {predicted_class}")


1/1 [==============================] - 1s 1s/step
The predicted class for the test image is: monster_energy_juice_aussie_lemonade


In [ ]:
import numpy as np
from PIL import Image

# Load the test image
test_image_path = '/content/drive/MyDrive/ListMissingProducts/test_miss.jpeg'  # Replace with your image path
img = Image.open(test_image_path)

# Ensure the image has 3 channels (RGB)
img = img.convert("RGB")

# Define sliding window parameters
window_size = (224, 224)  # Size similar to what the model expects
stride = 50  # Stride for the sliding window

# Perform sliding window operation
image_width, image_height = img.size
predictions = []

for y in range(0, image_height - window_size[1], stride):
    for x in range(0, image_width - window_size[0], stride):
        # Crop a window from the image
        window = img.crop((x, y, x + window_size[0], y + window_size[1]))

        # Preprocess the window similar to the training data
        window = window.resize((224, 224))  # Resize to the model's input size
        window = np.array(window) / 255.0  # Rescale pixel values
        window = np.expand_dims(window, axis=0)  # Add batch dimension

        # Make predictions using the trained model
        prediction = model.predict(window)
        predictions.append((x, y, inverse_mapping[np.argmax(prediction)]))

# Print predicted classes for each window
for pred in predictions:
    print(f"Class: {pred[2]}, Location: ({pred[0]}, {pred[1]})")


1/1 [==============================] - 0s 80ms/step
Class: monster_energy_lewis_hamilton, Location: (0, 0)
Class: monster_energy_lewis_hamilton, Location: (50, 0)
Class: monster_energy_lewis_hamilton, Location: (100, 0)
Class: monster_energy_lewis_hamilton, Location: (150, 0)
Class: monster_energy_lewis_hamilton, Location: (200, 0)
Class: monster_energy_zero_sugar_taurine, Location: (250, 0)
Class: monster_energy_mixxd, Location: (300, 0)
Class: monster_energy_mixxd, Location: (350, 0)
Class: monster_energy_zero_sugar_taurine, Location: (400, 0)
Class: monster_energy_lewis_hamilton, Location: (450, 0)
Class: monster_energy_lewis_hamilton, Location: (500, 0)
Class: monster_energy_lewis_hamilton, Location: (550, 0)
Class: monster_energy_lewis_hamilton, Location: (600, 0)
Class: monster_energy_lewis_hamilton, Location: (650, 0)
Class: monster_energy_original, Location: (700, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (750, 0)
Class: monster_energy_ultra_fiesta_mango, Location:

In [ ]:
# Implement non-maximum suppression (NMS)
def non_max_suppression(predictions, threshold=0.5):
    selected_predictions = []
    predictions.sort(key=lambda x: x[2], reverse=True)  # Sort predictions by confidence
    while len(predictions) > 0:
        selected_predictions.append(predictions[0])
        remaining = []
        for pred in predictions[1:]:
            iou = calculate_iou(selected_predictions[-1][:2], pred[:2])
            if iou < threshold:
                remaining.append(pred)
        predictions = remaining
    return selected_predictions

# Calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    # Calculate coordinates of intersection rectangle
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[0] + window_size[0], box2[0] + window_size[0])
    y_bottom = min(box1[1] + window_size[1], box2[1] + window_size[1])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = window_size[0] * window_size[1]
    box2_area = window_size[0] * window_size[1]
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou

# Apply NMS to consolidate predictions
consolidated_predictions = non_max_suppression(predictions, threshold=0.5)

# Print consolidated predictions
for pred in consolidated_predictions:
    print(f"Class: {pred[2]}, Location: ({pred[0]}, {pred[1]})")


NameError: ignored

Class: monster_energy_ultra_peachy_keen, Location: (0, 50)


In [ ]:
from scipy.spatial.distance import cdist

# Implement refined spatial clustering to merge nearby predictions selectively
def refined_clustering(predictions, distance_threshold=50):
    coords = np.array([[pred[0], pred[1]] for pred in predictions])
    dist_matrix = cdist(coords, coords)
    clustered_predictions = {}
    for i, pred in enumerate(predictions):
        close_preds = np.where(dist_matrix[i] < distance_threshold)[0]
        cluster_key = tuple(close_preds.tolist())
        if cluster_key not in clustered_predictions:
            clustered_predictions[cluster_key] = []
        clustered_predictions[cluster_key].append(pred)
    return [max(cluster, key=lambda x: x[2]) for cluster in clustered_predictions.values()]

# Apply refined clustering to consolidate predictions
consolidated_predictions = refined_clustering(predictions, distance_threshold=50)

# Print consolidated predictions
for pred in consolidated_predictions:
    print(f"Class: {pred[2]}, Location: ({pred[0]}, {pred[1]})")


Class: monster_energy_ultra, Location: (0, 0)
Class: monster_energy_ultra, Location: (50, 0)
Class: monster_energy_ultra, Location: (100, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (150, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (200, 0)
Class: monster_energy_original, Location: (250, 0)
Class: monster_energy_original, Location: (300, 0)
Class: monster_energy_original, Location: (350, 0)
Class: monster_energy_original, Location: (400, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (450, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (500, 0)
Class: monster_energy_juice_mango_loco, Location: (550, 0)
Class: monster_energy_ultra, Location: (600, 0)
Class: monster_energy_ultra, Location: (650, 0)
Class: monster_energy_ultra, Location: (700, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (750, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (800, 0)
Class: monster_energy_ultra_fiesta_mango, Location: (850, 0)
Class: monster_energy_ori

from collections import defaultdict

# Calculate distance from the center of the window
def calculate_distance(window):
    return abs(window[0] - 112) + abs(window[1] - 112)  # Assuming 224x224 window size

# Store unique predictions for each class based on closest distance to window center
unique_predictions = defaultdict(lambda: None)

# Process predictions to retain unique ones for each class
for pred in predictions:
    current_class = pred[2]
    if unique_predictions[current_class] is None or calculate_distance(pred) < calculate_distance(unique_predictions[current_class]):
        unique_predictions[current_class] = pred

# Print unique predictions
for pred in unique_predictions.values():
    print(f"Class: {pred[2]}, Location: ({pred[0]}, {pred[1]})")


In [ ]:
from collections import defaultdict

# Calculate distance from the center of the window
def calculate_distance(window):
    return abs(window[0] - 112) + abs(window[1] - 112)  # Assuming 224x224 window size

# Store unique predictions for each class based on closest distance to window center
unique_predictions = defaultdict(lambda: None)

# Process predictions to retain unique ones for each class
for pred in predictions:
    current_class = pred[2]
    if unique_predictions[current_class] is None or calculate_distance(pred) < calculate_distance(unique_predictions[current_class]):
        unique_predictions[current_class] = pred

# Print unique predictions
for pred in unique_predictions.values():
    print(f"Class: {pred[2]}, Location: ({pred[0]}, {pred[1]})")


Class: monster_energy_lewis_hamilton, Location: (50, 100)
Class: monster_energy_zero_sugar_taurine, Location: (250, 100)
Class: monster_energy_mixxd, Location: (150, 100)
Class: monster_energy_original, Location: (150, 450)
Class: monster_energy_ultra_fiesta_mango, Location: (50, 150)
Class: monster_energy_ultra, Location: (900, 450)
Class: monster_energy_juice_aussie_lemonade, Location: (600, 150)
Class: monster_energy_pipeline, Location: (100, 100)
Class: monster_energy_ultra_rosa, Location: (100, 150)
Class: monster_energy_nitro, Location: (0, 300)
Class: monster_energy_ultra_peachy_keen, Location: (950, 250)
Class: monster_energy_pacific_punch, Location: (50, 300)
Class: monster_energy_juice_monarch, Location: (100, 300)


TypeError: ignored